<a href="https://colab.research.google.com/github/KSundaram17/RAG_Take_Home_Assessment/blob/main/take_home_assessment_gpt_clean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from transformers import AutoModel, AutoTokenizer, GPT2LMHeadModel, GPT2Tokenizer
from sklearn.metrics.pairwise import cosine_similarity

# Load the sentence-transformers model for embedding
model_name = "sentence-transformers/all-mpnet-base-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# Load the GPT-2 model and tokenizer for generating responses
gpt2_model = GPT2LMHeadModel.from_pretrained('gpt2')
gpt2_tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
documents = [
    {
        "title": "Device Reset Guide",
        "content": "To reset your device, start by turning it off. Press and hold the power button for ten seconds. The device will automatically restart."
    },
    {
        "title": "Warranty Policy",
        "content": "Our products are guaranteed for one year from the purchase date. If your device malfunctions within this period, please contact our support center for a free repair or replacement."
    },
    {
        "title": "Software Update Procedures",
        "content": "To update your device's software, navigate to 'Settings', then 'System Updates'. Ensure your device is connected to WiFi. Click 'Check for updates'. If an update is available, select 'Download and install'."
    },
    {
        "title": "Troubleshooting Sound Issues",
        "content": "If your device's sound is not working, ensure the volume is turned up and that the device is not in silent mode. Check the sound settings in the 'Audio' section to adjust your preferences."
    },
    {
        "title": "Repair Service Process",
        "content": "To initiate a repair service, visit our website and fill out the repair request form. Include your device model and a detailed description of the issue. Ship your device to our service center using the provided prepaid shipping label."
    }
]

def chunk_text(text, chunk_size=100):
    """ Breaks text into chunks of specified size. """
    return [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]

# Prepare and chunk the documents
chunked_data = []
for doc in documents:
    chunks = chunk_text(doc['content'], 100)
    for chunk in chunks:
        chunked_data.append((doc['title'], chunk))  # Tuple of title and chunk


In [ ]:
def embed_text(text):
    inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt")
    outputs = model(**inputs)
    return outputs.pooler_output.squeeze(0).detach()

In [ ]:
import torch

# Embed each chunk using the previously defined `embed_text` function
document_embeddings = torch.stack([embed_text(chunk[1]) for chunk in chunked_data])


In [ ]:
def retrieve_most_relevant(query, embeddings, chunked_data):
    query_embedding = embed_text(query)
    if isinstance(query_embedding, torch.Tensor):
        query_embedding = query_embedding.cpu().numpy()
    if isinstance(embeddings, torch.Tensor):
        embeddings = embeddings.cpu().numpy()

    similarities = cosine_similarity([query_embedding], embeddings)
    most_relevant_idx = similarities.argmax()
    return chunked_data[most_relevant_idx]


In [ ]:
def generate_response(prompt):
    inputs = gpt2_tokenizer.encode(prompt, return_tensors='pt')
    outputs = gpt2_model.generate(inputs, max_length=200, no_repeat_ngram_size=2)
    return gpt2_tokenizer.decode(outputs[0], skip_special_tokens=True)

# Sample query
query = "How do I reset my device?"
title, relevant_chunk = retrieve_most_relevant(query, document_embeddings, chunked_data)

# Create a more comprehensive prompt
prompt = f"{title}: {relevant_chunk} Please provide step-by-step instructions."
response = generate_response(prompt)
print("Response:", response)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Response: Device Reset Guide: To reset your device, start by turning it off. Press and hold the power button for ten seconds. The  Please provide step-by-step instructions.

Step 2: Turn off your phone
. If you have a phone that is not connected to the internet, you can turn off the phone by pressing and holding the Power button. You can also turn it on by holding and releasing the Volume key. Once the device is turned on, press and release the volume key for 10 seconds, then press the Reset button to reset the computer.


If you are using a USB cable, it is recommended to use a power adapter. This will allow you to connect your computer to your USB port. To connect a computer, simply connect the USB to a PC or Mac.


In [ ]:
# Ensure models and tokenizers are loaded
gpt2_model = GPT2LMHeadModel.from_pretrained('gpt2')
gpt2_tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Embed, retrieve, and respond
document_embeddings = torch.stack([embed_text(chunk[1]) for chunk in chunked_data])
title, relevant_chunk = retrieve_most_relevant("How do I update the software?", document_embeddings, chunked_data)
prompt = f"{title}: {relevant_chunk} What are the next steps?"
response = generate_response(prompt)
print("Response:", response)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Response: Software Update Procedures: To update your device's software, navigate to 'Settings', then 'System Updates'. Ensure your device  What are the next steps?

Step 1: Download and install the latest version of Windows 10.
...
 (Note: If you are using Windows 7 or Windows 8.1, you will need to download and run the Windows Update tool from the Microsoft Download Center.)
,. (If you're using Microsoft Windows, then you'll need the following:
- Windows Server 2012 R2
 - Windows Phone 8
The following steps are required to install Windows Updates: Install the update from Microsoft's Download Manager. Download the Update Manager from here. Install Windows updates from your computer. Open the Updates tab. Click the 'Install' button. Select the version you want to update. Choose the one you wish to upgrade. If the upgrade is not available, click 'Yes'. Click 'Next'.
: Click OK. You should see the download


In [ ]:
pip install elasticsearch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 477.5/477.5 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.3/64.3 kB 6.7 MB/s eta 0:00:00


In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk

es = Elasticsearch()

def create_index(index_name):
    # Create an index with a dense_vector type for embeddings
    es.indices.create(index=index_name, body={
        "settings": {"number_of_shards": 1},
        "mappings": {
            "properties": {
                "title": {"type": "text"},
                "chunk": {"type": "text"},
                "embedding": {"type": "dense_vector", "dims": 768}
            }
        }
    })

def index_documents(chunked_data):
    actions = [
        {
            "_index": "document_index",
            "_source": {
                "title": title,
                "chunk": chunk,
                "embedding": embed_text(chunk)
            }
        } for title, chunk in chunked_data
    ]
    bulk(es, actions)

# Create index and index documents
create_index("document_index")
index_documents(chunked_data)


In [ ]:
#wRead it from reshift tables
import psycopg2

def load_data_from_redshift():
  try:
    connection = psycopg2.connect(
        db_name = database_name,
        user = user_id,
        password = password
        host = host_name,
        port = port_name
    )
    cursor = connection.cursor()
    query = "SELECT * FROM document_table"
    cursor.execute(query)
    rows = cursor.fetchall()
    return rows
  except Exception as  error:
    print("Error is: ", error)

  finally:
    if connection:
      cursor.close()
      connection.close()

data =  load_data_from_redshift()


In [ ]:
# Wriet the data tp redshift
import psycopg2
from datetime import datetime

# Connection parameters
params = {
    'database': redshift_databse,
    'user': user_id,
    'password': password,
    'host': host_name,
    'port': '5439'
}

# Create a function to insert data
def insert_question_answer(question, answer):
    try:
        conn = psycopg2.connect(**params)
        cursor = conn.cursor()
        insert_query = """
        INSERT INTO qa_table (QuestionText, AnswerText, CreatedAt)
        VALUES (%s, %s, %s);
        """
        cursor.execute(insert_query, (question, answer, datetime.now()))
        conn.commit()
        cursor.close()
        conn.close()
        print("Inserted successfully")
    except Exception as e:
        print("Error: ", e)
        if conn:
            conn.rollback()
            cursor.close()
            conn.close()

# Example usage
insert_question_answer("How do I reset my device?", "To reset your device, start by turning it off and then...")
